In [1]:
import cv2
import mediapipe as mp
import tkinter as tk
from tkinter import messagebox
import time

root=tk.Tk()
root.withdraw()
face_cascade=cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

alert=False
photo=False
poptime=0

def phototaken(video_data,face_detect):
    global photo
    for (x,y,w,h) in face_detect:
        faceimg=video_data[y:y+h,x:x+w]
        f=f"face_{int(time.time())}.jpg"
        cv2.imwrite(f,faceimg)
        print(f"filed saved as {f}")
        photo=True
        break
        


    
def showalert():
    global alert,poptime
    alert=True
    poptime=time.time()
    messagebox.showinfo("alert","full face detected")

mp_hands=mp.solutions.hands
mp_drawing=mp.solutions.drawing_utils
hands = mp_hands.Hands(max_num_hands=4,min_detection_confidence=0.7)
video_cap = cv2.VideoCapture(0)
while True:
    ret,video_data=video_cap.read()
    if not ret :
        break

    video_data=cv2.flip(video_data,1)
    rgb=cv2.cvtColor(video_data,cv2.COLOR_BGR2RGB)
    result=hands.process(rgb)
    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            mp_drawing.draw_landmarks(video_data,hand_landmarks,mp_hands.HAND_CONNECTIONS)

    gray_image=cv2.cvtColor(video_data,cv2.COLOR_BGR2GRAY)
    face_detect=face_cascade.detectMultiScale(gray_image,scaleFactor=1.1,minNeighbors=8)
    
    if len(face_detect)>0 :
        for (x,y,w,h) in face_detect:
            if w>150 and h>150:
                if time.time() - poptime >=5 :
                    showalert()
                    if not photo:
                        phototaken(video_data,face_detect)

            cv2.rectangle(video_data,(x,y),(x+w,y+h),(0,255,255),1)

    elif len(face_detect)==0:
        alert=False
        photo=False
    
    cv2.imshow("hands detection live",video_data)

    if cv2.waitKey(1) & 0xFF == ord(" "):
       break

video_cap.release()
cv2.destroyAllWindows()


filed saved as face_1755241140.jpg
filed saved as face_1755241142.jpg
filed saved as face_1755241148.jpg
